In [ ]:
!pip install scikit-fuzzy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 994.0/994.0 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-fuzzy: filename=scikit_fuzzy-0.4.2-py3-none-any.whl size=894078 sha256=fe6e41f4433637f71517c78f66354861e728e3dc3d994ec1d15ea3843ecd27c3
  Stored in directory: /root/.cache/pip/wheels/4f/86/1b/dfd97134a2c8313e519bcebd95d3fedc7be7944db022094bc8
Successfully built scikit-fuzzy


In [ ]:
import pandas as pd
import numpy as np
import skfuzzy as fuzz
import skfuzzy.membership as mf
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import model_selection
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/csv/data.csv", delimiter=";") #Memanggil data yg ada di file data csv
df.head() #menampilkan 5 data teratas

,Marital status,Application mode,Application order,Course,Daytime/evening attendance\t,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,1,17,5,171,1,1,122.0,1,19,12,...,0,0,0,0,0.000000,0,10.8,1.4,1.74,Dropout
1,1,15,1,9254,1,1,160.0,1,1,3,...,0,6,6,6,13.666667,0,13.9,-0.3,0.79,Graduate
2,1,1,5,9070,1,1,122.0,1,37,37,...,0,6,0,0,0.000000,0,10.8,1.4,1.74,Dropout
3,1,17,2,9773,1,1,122.0,1,38,37,...,0,6,10,5,12.400000,0,9.4,-0.8,-3.12,Graduate
4,2,39,1,8014,0,1,100.0,1,37,38,...,0,6,6,6,13.000000,0,13.9,-0.3,0.79,Graduate


In [ ]:
inputs = df.drop(df.columns[[0,1,2,3,4,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,26,27,28,29,30,32,33,34,35,36]], axis=1)
target = df.Target

inputs.head(15)

,Previous qualification,Previous qualification (grade),Curricular units 1st sem (grade),Curricular units 2nd sem (grade)
0,1,122.0,0.000000,0.000000
1,1,160.0,14.000000,13.666667
2,1,122.0,0.000000,0.000000
3,1,122.0,13.428571,12.400000
4,1,100.0,12.333333,13.000000
5,19,133.1,11.857143,11.500000
6,1,142.0,13.300000,14.345000
7,1,119.0,0.000000,0.000000
8,1,137.0,13.875000,14.142857
9,1,138.0,11.400000,13.500000


In [ ]:
target.head(15)

0      Dropout
1     Graduate
2      Dropout
3     Graduate
4     Graduate
5     Graduate
6     Graduate
7      Dropout
8     Graduate
9      Dropout
10    Graduate
11    Graduate
12     Dropout
13    Graduate
14    Graduate
Name: Target, dtype: object

In [ ]:
inputs.columns[inputs.isna().any()]

Index([], dtype='object')

In [ ]:
# Bagi dataset menjadi data latih dan data uji
x_train, x_test, y_train, y_test = train_test_split(inputs, target, test_size = 0.3, random_state = 100)

In [ ]:
# Fuzzifikasi atribut numerik menjadi himpunan fuzzy
# Di sini kita menggunakan fungsi keanggotaan segitiga untuk setiap atribut

# X_train
x_train_array = x_train.values
x_train_fuzzy = []
for i in range(x_train.shape[1]):
    attr_fuzzy = []
    attr_min = np.min(x_train_array[:, i])
    attr_max = np.max(x_train_array[:, i])
    attr_fuzzy.append(fuzz.trimf(x_train_array[:, i], [attr_min, attr_min, attr_max])) #Menggunakan fungsi keanggotaan segitiga fuzzy
    x_train_fuzzy.append(attr_fuzzy)

# X_test
x_test_array = x_test.values
x_test_fuzzy = []
for i in range(x_test.shape[1]):
    attr_fuzzy = []
    attr_min = np.min(x_test_array[:, i])
    attr_max = np.max(x_test_array[:, i])
    attr_fuzzy.append(fuzz.trimf(x_test_array[:, i], [attr_min, attr_min, attr_max]))
    x_test_fuzzy.append(attr_fuzzy)


In [ ]:
# Mengembalikan x_train_fuzzy ke tipe data numpyarray kembali
x_train_restored = np.array(x_train_fuzzy).squeeze().T
x_train_restored[0:10]

print('Hasil Fungsi Keangotaan X_Train')
pd.DataFrame(x_train_restored, columns=['Previous qualification', 'Previous qualification (grade)',
                                     'Curricular units 1st sem (grade)', 'Curricular units 2nd sem (grade)'])

Hasil Fungsi Keangotaan X_Train


,Previous qualification,Previous qualification (grade),Curricular units 1st sem (grade),Curricular units 2nd sem (grade)
0,1.0,0.505263,0.183333,0.208462
1,1.0,0.536842,0.256349,0.279135
2,1.0,0.873684,0.333333,0.300000
3,1.0,0.526316,0.271605,0.369231
4,1.0,0.621053,0.301587,0.374038
...,...,...,...,...
3091,1.0,0.452632,1.000000,1.000000
3092,1.0,0.578947,1.000000,1.000000
3093,1.0,0.473684,0.235486,0.259010
3094,1.0,0.505263,1.000000,1.000000


In [ ]:
# Mengembalikan x_test_fuzzy ke data kembali
x_test_restored = np.array(x_test_fuzzy).squeeze().T
x_test_restored[0:10]
print('Hasil Fungsi Keangotaan X_Test')
pd.DataFrame(x_test_restored, columns=['Previous qualification', 'Previous qualification (grade)',
                                     'Curricular units 1st sem (grade)', 'Curricular units 2nd sem (grade)'])

Hasil Fungsi Keangotaan X_Test


,Previous qualification,Previous qualification (grade),Curricular units 1st sem (grade),Curricular units 2nd sem (grade)
0,1.000000,0.558140,0.311258,0.212806
1,1.000000,0.500000,0.311258,1.000000
2,1.000000,0.732558,0.427815,0.406522
3,1.000000,0.395349,0.311258,0.274638
4,1.000000,0.593023,0.296121,0.265217
...,...,...,...,...
1323,0.571429,0.568605,0.302428,0.265217
1324,1.000000,0.604651,0.403974,0.406522
1325,0.571429,0.953488,0.293598,0.236957
1326,0.952381,0.372093,0.106906,0.067391


In [ ]:
model = GaussianNB()

model_train = model.fit(x_train_restored, y_train)

In [ ]:
pd.DataFrame(x_test_restored[0:10], columns=['Previous qualification', 'Previous qualification (grade)',
                                     'Curricular units 1st sem (grade)', 'Curricular units 2nd sem (grade)'])


,Previous qualification,Previous qualification (grade),Curricular units 1st sem (grade),Curricular units 2nd sem (grade)
0,1.000000,0.558140,0.311258,0.212806
1,1.000000,0.500000,0.311258,1.000000
2,1.000000,0.732558,0.427815,0.406522
3,1.000000,0.395349,0.311258,0.274638
4,1.000000,0.593023,0.296121,0.265217
5,0.952381,0.488372,0.169499,0.113981
6,1.000000,0.569767,0.355408,0.227536
7,1.000000,0.604651,0.364238,0.287826
8,0.952381,0.604651,0.402769,0.321739
9,1.000000,0.488372,0.434879,0.359420


In [ ]:
# Menentukan hasil prediksi dari x_test_fuzzy
y_pred = model_train.predict(x_test_restored)

print('Hasil Prediksi')
print('==================\n')
pd.DataFrame(y_pred[0:10], columns=['label'])


Hasil Prediksi



,label
0,Graduate
1,Dropout
2,Graduate
3,Graduate
4,Graduate
5,Graduate
6,Graduate
7,Graduate
8,Graduate
9,Graduate


In [ ]:
print('Sebelum Prediksi')
print('==================\n')
pd.DataFrame(np.array(y_test[0:10]), columns=['label'])

Sebelum Prediksi



,label
0,Graduate
1,Dropout
2,Enrolled
3,Graduate
4,Graduate
5,Graduate
6,Graduate
7,Graduate
8,Graduate
9,Dropout


In [ ]:
predictResult = model_train.predict_proba(x_test_restored[0:10])

print('Hasil Probabilitas Prediksi')
print('====================================\n')
pd.DataFrame(predictResult, columns=['Dropout', 'Enrolled', 'Graduate'])

Hasil Probabilitas Prediksi



,Dropout,Enrolled,Graduate
0,0.033749,0.115059,0.851191
1,0.969040,0.030473,0.000488
2,0.107800,0.236826,0.655373
3,0.025926,0.085464,0.888610
4,0.034886,0.120932,0.844182
5,0.034504,0.099039,0.866457
6,0.039733,0.129060,0.831207
7,0.045835,0.144706,0.809459
8,0.060694,0.169019,0.770287
9,0.069115,0.168966,0.761919


In [ ]:
confusionMatrix = confusion_matrix(y_test, y_pred)
confusionMatrix

print('Confusion Matrix')
print('================================= \n')
pd.DataFrame(confusionMatrix, columns=['Dropout', 'Enrolled', 'Graduate'], index=['Dropout', 'Enrolled', 'Graduate'])


Confusion Matrix



,Dropout,Enrolled,Graduate
Dropout,208,3,191
Enrolled,24,1,221
Graduate,20,0,660


In [ ]:
print('Tingkat Akurasi')
print('======================================================= \n')
print(classification_report(y_test, y_pred))

Tingkat Akurasi

              precision    recall  f1-score   support

     Dropout       0.83      0.52      0.64       402
    Enrolled       0.25      0.00      0.01       246
    Graduate       0.62      0.97      0.75       680

    accuracy                           0.65      1328
   macro avg       0.56      0.50      0.47      1328
weighted avg       0.61      0.65      0.58      1328

